# Imports

In [289]:
import pandas as pd
import numpy as np
import json
import datetime
import ast

from sklearn import preprocessing as pp
import pickle

from sklearn import model_selection as ms
import category_encoders as ce

#from boruta   import BorutaPy
from lightgbm import LGBMClassifier

from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

## Helper Functions

In [290]:
def test_preparation(X_test, y_test):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test, y_test


def rescalling (X_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    X_test['avg_delivery_time_days'] = mm.transform(X_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    
    X_test['average_rating'] = rs_ar.transform(X_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    
    X_test['number_of_reviews'] = rs_nr.transform(X_test[['number_of_reviews']])
    
    return X_test

def transform (X_test):
    
    for att in ['brand', 'product', 'seller']:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        X_test = fe.transform(X_test)
    
    return X_test

def log_var_resposta(df):
    df = np.log1p(df)
    return df

def feature_selection(df):
    features_selected = [
    'average_rating',
    'number_of_reviews',
    'brand',
    # 'category',
    # 'crawled_at',
    'out_of_stock',
    'avg_delivery_time_days',
    # 'product_details',
    'seller',
    # 'sub_category',
    # 'fabrication_time',
    # 'title',
    'product'
    ]
    
    return df[features_selected]


# Load Data

In [291]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as 
# a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10 21:17:28,Navy Blue Printed Boxers Has An Inner Elasti...,['https://rukminim1.flixcart.com/image/128/128...,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11 01:02:46,smiley printed tshirt on round neck cotton tshirt,['https://rukminim1.flixcart.com/image/128/128...,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11 00:43:37,Free Authority Presents this Crew Neck Yellow ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10 21:22:10,Cotton Blend FabricCollar / Polo Neck White & ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10 23:36:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


# Data Description

In [292]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [293]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [294]:
data_train.dtypes

_id                        object
average_rating            float64
number_of_reviews           int64
brand                      object
category                   object
crawled_at                 object
out_of_stock                 bool
avg_delivery_time_days      int64
pid                        object
product_details            object
seller                     object
sub_category               object
fabrication_time            int64
title                      object
actual_price              float64
dtype: object

In [295]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [296]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [297]:
data_train = data_train.dropna(subset=['actual_price'])

In [298]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.0
average_rating            0.0
number_of_reviews         0.0
brand                     0.0
category                  0.0
crawled_at                0.0
out_of_stock              0.0
avg_delivery_time_days    0.0
pid                       0.0
product_details           0.0
seller                    0.0
sub_category              0.0
fabrication_time          0.0
title                     0.0
actual_price              0.0
dtype: float64

## Entendendo product Details

In [299]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

teste = data_train['product_details'].apply(lambda x: f(x))

def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario


df = pd.DataFrame([junta_dict(row) for row in teste])

In [300]:
df = pd.DataFrame([junta_dict(row) for row in teste])
df.isna().sum().sort_values()

Fabric               1491
Style Code           1653
Pattern              1661
Fabric Care          4305
Suitable For         6120
                    ...  
Shoulder in inch    19811
Pleated             19812
Region              19812
Design              19812
Fabric care         19812
Length: 124, dtype: int64

# Feature Engineering

In [301]:
df4 = data_train.copy()

In [302]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros
df4['brand'] = df4['brand'].apply(lambda x: x.lower())

# criando feature product
df4['product'] = df4[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)

In [303]:
## Tratando variaveis respostas nulas
prices = df4[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()

precos_nulos = df4.loc[df4['actual_price'].isna(), ['_id','pid']]

precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

df4_aux = pd.merge(df4, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
df4['actual_price'] = df4_aux['actual_price_x'].fillna(0) + df4_aux['actual_price_y'].fillna(0)

df4 = df4.dropna(subset=['actual_price'])

# EDA

In [304]:
df5 = df4.copy()

# Data Preparation

In [305]:
df6 = df5.copy()

In [306]:
df6.isna().sum()

_id                       0
average_rating            0
number_of_reviews         0
brand                     0
category                  0
crawled_at                0
out_of_stock              0
avg_delivery_time_days    0
pid                       0
product_details           0
seller                    0
sub_category              0
fabrication_time          0
title                     0
actual_price              0
product                   0
dtype: int64

## Split Dataset

In [307]:
X = df6.drop(['_id','pid','actual_price'], axis=1)
y = df6['actual_price']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.01)
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

## Rescalling

In [308]:
X_train.columns

Index(['average_rating', 'number_of_reviews', 'brand', 'category',
       'crawled_at', 'out_of_stock', 'avg_delivery_time_days',
       'product_details', 'seller', 'sub_category', 'fabrication_time',
       'title', 'product'],
      dtype='object')

### Min Max Scaler

In [309]:
mm = pp.MinMaxScaler()

X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])
pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/3312916923.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])


### Robust Scaler

In [310]:
rs_average_rating   = pp.RobustScaler()
rs_number_of_reviews = pp.RobustScaler()

X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])
pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))

C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/3765211346.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/3765211346.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])


## Transformation

### One Hot Enconde

In [311]:
# 'out_of_stock'

### Target Encode

In [ ]:
# for att in []:
#     te = pickle.load(open(f'api/parameters/te{att}.pkl','rb'))
#     X_train =te.transform(X_train)

### Frequency Encode

In [312]:
for att in ['brand', 'product', 'seller']:
    fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
    pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
    X_train = fe.transform(X_train)

### Response Variable

In [313]:
y_train = np.log1p(y_train)

## Test Preparation

In [314]:
X_test_mod, y_test_mod = test_preparation(X_test, y_test)

C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/2537213164.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['avg_delivery_time_days'] = mm.transform(X_test[['avg_delivery_time_days']])
C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/2537213164.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['average_rating'] = rs_ar.transform(X_test[['average_rating']])
C:\Users\Notebook\AppData\Local\Temp/ipykernel_3748/2537213164.py:20: SettingWithCopyWarning: 
A value is trying to be set on

# Feature Selection

In [315]:
X_train.columns.tolist()

['average_rating',
 'number_of_reviews',
 'brand',
 'category',
 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
 'product_details',
 'seller',
 'sub_category',
 'fabrication_time',
 'title',
 'product']

In [316]:
features_selected = [
'average_rating',
'number_of_reviews',
'brand',
# 'category',
# 'crawled_at',
'out_of_stock',
'avg_delivery_time_days',
# 'product_details',
'seller',
# 'sub_category',
# 'fabrication_time',
# 'title',
'product'
]

## Importancia das árvores

In [317]:
# Crie e treine o modelo LGBMClassifier
lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
X_train_sel = X_train[features_selected]
lgb_model.fit(X_train_sel, y_train)

# Obtenha as importâncias das características do modelo
importances = lgb_model.feature_importances_

# Ordene as características por importância decrescente
indices = np.argsort(importances)[::-1]

# Imprima o ranking das características
print('Feature ranking')
df = pd.DataFrame()

for i, j in zip(X_train_sel.columns, importances):
    aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])
    df = pd.concat([df, aux], axis=0)

print(df.sort_values('importance', ascending=False))

Feature ranking
                  feature  importance
0          average_rating         989
0                   brand         906
0                  seller         646
0                 product         434
0  avg_delivery_time_days          14
0            out_of_stock          11
0       number_of_reviews           0


# Machine Learning

In [336]:
def MachineLearning(X_train, y_train, X_test, y_test, features_selected):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=50, max_depth=3),
                        SGDRegressor(random_state=SEED), 
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'SGD', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    
    X_train_selected = X_train[features_selected]
    X_test_selected = X_test[features_selected]


    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [337]:
MachineLearning(X_train, y_train, X_test_mod, y_test_mod, features_selected)

Rodando modelo: XGBoost
Rodando modelo: SGD
Rodando modelo: LGBM
Rodando modelo: DecisionTree
Rodando modelo: KNN
Rodando modelo: Bagging
Rodando modelo: RandomForest


,SMAPE
XGBoost,40.970708
SGD,41.257167
LGBM,40.985862
DecisionTree,43.695421
KNN,44.259838
Bagging,43.098499
RandomForest,42.975662


# Fine Tuning

- Não possui cross validation
- Não realiza reencode das variáveis
- **Para maior assertividade, rodar com todos os dados de treino disponíveis**

In [362]:
def objective_cv(trial):  
    
    n_estimators = trial.suggest_int('n_estimators',20, 100)
    eta = trial.suggest_float('eta',0.01, 0.05)
    max_depth = trial.suggest_int('max_depth', 3, 9)
    subsample = trial.suggest_float('subsample',0.1, 0.9)
    colsample_bytree = trial.suggest_float('colsample_bytree',0.1,0.9)

    # model definition
    xgb_model = XGBRegressor( n_estimators    = n_estimators, 
                              eta              = eta, 
                              max_depth        = max_depth, 
                              subsample        = subsample,
                              colsample_bytree = colsample_bytree
                             )
    
    xgb_model.fit(X_train[features_selected], y_train)
    
    # Model Prediction 
    y_pred = xgb_model.predict(X_test_mod[features_selected])

    # calculando rmse
    smape = 100 / len(y_test_mod) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test_mod)) / (np.abs(np.expm1(y_test_mod)) + np.abs(np.expm1(y_pred))))

    return smape

In [363]:
import optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective_cv, n_trials=50)

[I 2023-10-21 18:03:58,536] A new study created in memory with name: no-name-83c3fc56-7673-4526-a67b-ba759b289ec5
[I 2023-10-21 18:03:59,040] Trial 0 finished with value: 43.50519765794977 and parameters: {'n_estimators': 77, 'eta': 0.04499951251092036, 'max_depth': 5, 'subsample': 0.3523407370384566, 'colsample_bytree': 0.16439402033866113}. Best is trial 0 with value: 43.50519765794977.
[I 2023-10-21 18:03:59,251] Trial 1 finished with value: 177.58046308442437 and parameters: {'n_estimators': 38, 'eta': 0.021730646031018416, 'max_depth': 7, 'subsample': 0.1304730163838916, 'colsample_bytree': 0.42254313509059727}. Best is trial 0 with value: 43.50519765794977.
[I 2023-10-21 18:03:59,452] Trial 2 finished with value: 117.5849552201402 and parameters: {'n_estimators': 33, 'eta': 0.04651253268253888, 'max_depth': 7, 'subsample': 0.8208510970670676, 'colsample_bytree': 0.3243842469364089}. Best is trial 0 with value: 43.50519765794977.
[I 2023-10-21 18:03:59,695] Trial 3 finished with v

[I 2023-10-21 18:04:23,490] Trial 29 finished with value: 44.04242747369732 and parameters: {'n_estimators': 82, 'eta': 0.041278012760988976, 'max_depth': 5, 'subsample': 0.8958770782013493, 'colsample_bytree': 0.6764666267346452}. Best is trial 24 with value: 41.50543641531861.
[I 2023-10-21 18:04:24,135] Trial 30 finished with value: 56.96812771541128 and parameters: {'n_estimators': 56, 'eta': 0.04581524152224532, 'max_depth': 6, 'subsample': 0.6354206444938022, 'colsample_bytree': 0.7808527346383008}. Best is trial 24 with value: 41.50543641531861.
[I 2023-10-21 18:04:25,146] Trial 31 finished with value: 41.82479416547906 and parameters: {'n_estimators': 94, 'eta': 0.04722128796837078, 'max_depth': 6, 'subsample': 0.5946654630004923, 'colsample_bytree': 0.7744943591087431}. Best is trial 24 with value: 41.50543641531861.
[I 2023-10-21 18:04:26,365] Trial 32 finished with value: 41.92472377220769 and parameters: {'n_estimators': 96, 'eta': 0.04627205103531657, 'max_depth': 7, 'subs

In [364]:
best_params = study.best_params
best_params

{'n_estimators': 96,
 'eta': 0.04992218621306727,
 'max_depth': 3,
 'subsample': 0.847860379813958,
 'colsample_bytree': 0.42739526121562105}

In [365]:
best_model = XGBRegressor(n_jobs = -1 , 
                                 random_state=0,
                                 n_estimators = best_params['n_estimators'],
                                 eta = best_params['eta'],
                                 max_depth = best_params['max_depth'],
                                 subsample = best_params['subsample'],
                                 colsample_bytree = best_params['colsample_bytree'])

# model fit
best_model.fit(X_train[features_selected], y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.42739526121562105, early_stopping_rounds=None,
             enable_categorical=False, eta=0.04992218621306727,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=96, n_jobs=-1, num_parallel_tree=None, predictor=None, ...)

## Submission

### Carregar os dados

In [366]:
# Opening JSON file
sub = open('dataset/raw/test.json')
 
# returns JSON object as 
# a dictionary
data_sub_aux = json.load(sub)

data_sub = pd.json_normalize(data_sub_aux, record_path = 'data')
data_sub.columns = data_sub_aux['columns']
data_sub.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title
0,35d28961-170f-515d-90d4-55ee978a8afa,4.8,40,Mo,Clothing and Accessories,2021-02-10 21:18:47,Accentuate your formal wardrobe by buying this...,['https://rukminim1.flixcart.com/image/128/128...,False,17,SHTFDYZGEXPED7YS,"[{'Pack of': '1'}, {'Model Name': 'Mods Slim f...",KKSONS,Topwear,32,Men Slim Fit Printed Button Down Collar Casual...
1,5089618f-7eec-571b-84b6-41c3b43a2ed4,3.7,31,True Bl,Clothing and Accessories,2021-02-10 22:12:20,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,10,TROFM47EVWRYK7ZU,"[{'Fit': 'Slim Fit'}, {'Occasion': 'Casual'}, ...",KAPSONSRETAILPVTLTD,Bottomwear,318,Slim Fit Men Brown Cotton Blend Trousers
2,84716375-2ea3-5376-bca1-ecef76b0aa87,3.2,26,vims rai,Clothing and Accessories,2021-02-11 01:16:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,8,FABFZAMPGDW3BKGX,"[{'Type': 'Shirt Fabric'}, {'Sales Package': '...",WHITE SKY,Fabrics,411,Rayon Printed Shirt Fabric (Unstitched)
3,84120873-2b67-5f2c-9074-803e52e278f7,3.6,30,True Bl,Clothing and Accessories,2021-02-10 22:21:32,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,10,BZRFNMGPNBQTYHMG,"[{'Color': 'Blue'}, {'Fabric': 'Polyester'}, {...",KAPSONSRETAILPVTLTD,"Blazers, Waistcoats and Suits",431,Solid Single Breasted Formal Men Full Sleeve B...
4,96238f84-3bb8-5ec1-b2f4-fe481ba35faa,2.5,21,ECKO Unl,Clothing and Accessories,2021-02-10 22:32:00,ECKO Unltd YD CHECK Cotton Woven Slim Fit IND...,['https://rukminim1.flixcart.com/image/128/128...,False,5,SHTFV5G6PZDQZEVZ,"[{'Pack of': '1'}, {'Style Code': 'EKSH001497'...",SandSMarketing,Topwear,512,Men Slim Fit Checkered Cut Away Collar Casual ...


### Transformar os dados

In [367]:
def limpeza (df):
    df = df.drop(columns=['_id', 'pid','description', 'images'])
    # transformando de string para data
    df['crawled_at'] = pd.to_datetime(df['crawled_at']).dt.date

    return df

def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    return df

def submission_preparation(df, features_selected):
    
    df = limpeza(df)
    df = feature_engen(df)
    df = rescalling(df)
    df = transform(df)
    
    return df[features_selected]

### Aplicar sobre os dados de teste

In [368]:
data_sub_mod = submission_preparation(data_sub, features_selected)

In [369]:
y_hat = np.expm1(best_model.predict(data_sub_mod))

In [370]:
df_submission = pd.DataFrame()
df_submission['pid'] = data_sub['pid']
df_submission['actual_price'] = y_hat

In [371]:
df_submission.to_csv('submissions/submission_4.csv', index=False)

------
# 10 Help Functions

In [327]:
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

In [328]:
def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

In [329]:
def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    ## Tratando variaveis respostas nulas
    prices = df[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()
    precos_nulos = df.loc[df['actual_price'].isna(), ['_id','pid']]
    precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')
    
    df_aux = pd.merge(df, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
    df['actual_price'] = df_aux['actual_price_x'].fillna(0) + df_aux['actual_price_y'].fillna(0)
    
    df = df.dropna(subset=['actual_price'])
    
    return df

In [330]:
def pipeline (df):
    df = limpeza(df)
    df = feature_engen(df)
    
    product_details = df['product_details'].apply(lambda x: f(x))
    product_details = pd.DataFrame([junta_dict(row) for row in product_details])
    
    return df, product_details


In [331]:
def test_preparation(df):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test

In [332]:
# '_id'
# 'average_rating':         muitos outliers mas distribuição proxima da normal
#                           tratar notas que não possuem avaliação
# 'number_of_reviews':      muitos outliers mas distribuição proxima da normal
#                           ver como fica a distribuição como log
# 'brand':                  12 categorias frequency -> transformar tudo em minusculo para evitar duplicações
# 'category':               Uma categoria é mais majoritária 
#                           unir com a subcategory fazendo o tratamento de maiusculas
# 'crawled_at':             Somente 1 dia, muda somente a hora -> retirar XXXXX
# 'out_of_stock':           Dummie enconde (somente 0 ou 1) -> One hot encode
# 'avg_delivery_time_days': sem outliers e distribuição normal -> min max scaler
# 'pid':                    valore duplicatidos, muitas labels -> usar para preencher valores nulos no price mas não vai para o modelo
# 'seller':                 Reunir undefined em "outros" e usar o frequency encode
# 'sub_category':           Unir com o 'category'
# 'fabrication_time':       sem outliers, distribuição uniforme -> min max scaler
# 'title':                  muitas categorias, sem concentrações -> frequency 
#                           explorar mais o title
#                           deletar nesse momento
# 'actual_price':           transformar em escala log